In [ ]:
# Cell 1: Setup
%load_ext autoreload
%autoreload 2

import sys
import gc
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score

# Setup path
sys.path.append(str(Path.cwd().parent))
from src import config, preprocessing, autoencoder, rf_classifier, evaluation, utils

# Create separate folder for Cross-Test results
exp_paths = utils.setup_experiment_folder()
print(f"Saving results to: {exp_paths['root']}")

📂 New Experiment Created: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_071127
📂 Saving results to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_071127


In [ ]:
# ==============================================================================
# SCENARIO 1: TRAIN ON 2017 -> TEST ON 2018
# ==============================================================================

# Cell 2: Load Data 2017 (TRAIN)
print("\n--- 1. LOADING TRAIN SET (CIC-IDS2017) ---")
# This function returns DataFrame, we keep it as is for easier control initially
df_train, y_train = preprocessing.load_single_dataset_year('2017', binary_mode=True)

# Convert to numpy for model input
X_train = df_train.values

print(f"Training Data Shape: {X_train.shape}")
print(f"Label Distribution: {pd.Series(y_train).value_counts()}")


--- 1. LOADING TRAIN SET (CIC-IDS2017) ---
🔄 Loading dataset year 2017 (Binary=True)...
✅ Loaded 2017. Shape: (2830743, 65)
Training Data Shape: (2830743, 65)
Label Distribution: Label
0    2273097
1     557646
Name: count, dtype: int64


In [ ]:
# Cell 3: Scaling (Fit on Train)
print("\n--- 2. SCALING ---")
scaler = preprocessing.get_scaler()
X_train_scaled = scaler.fit_transform(X_train)
print("Scaler fitted on 2017.")


--- 2. SCALING ---
✅ Scaler fitted on 2017.


In [ ]:
# Cell 4: Train Autoencoder on 2017
print(f"\n--- 3. TRAIN AUTOENCODER ---")

# 1. Define L correctly from config (L=30)
L = 30 

# 2. IMPORTANT: Slice scaled data to only take first 30 features
# X_train_scaled currently has 65 columns, we only take :L (0 to 29)
X_train_ae_input = X_train_scaled[:, :L] 

print(f"Actual input for AE: {X_train_ae_input.shape}") # Must be (N, 30)

# 3. Initialize and train
ae_model = autoencoder.DeepAutoencoder(
    input_dim=L,             # Match 30
    latent_dim=config.AE_LATENT_DIM,
    hidden_layers=config.AE_HIDDEN_LAYERS
)

ae_save_path = exp_paths['models'] / "ae_model_2017.pth"

# Pass X_train_ae_input instead of X_train_scaled
ae_model = autoencoder.train_ae(ae_model, X_train_ae_input, save_path=ae_save_path)

print("AE training done with 30 features.")


--- 3. TRAIN AUTOENCODER ---
Đầu vào thực tế cho AE: (2830743, 30)
[Autoencoder] Training on cuda...
  Epoch 5/50 - Loss: 0.269135
  Epoch 10/50 - Loss: 0.176908
  Epoch 15/50 - Loss: 0.151860
  Epoch 20/50 - Loss: 0.138725
  Epoch 25/50 - Loss: 0.128428
  Epoch 30/50 - Loss: 0.121462
  Epoch 35/50 - Loss: 0.118857
  Epoch 40/50 - Loss: 0.114712
  Epoch 45/50 - Loss: 0.112560
  Epoch 50/50 - Loss: 0.108991
✅ AE Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_071127\models\ae_model_2017.pth
✅ AE training done with 30 features.
Extracting latent features for Train set...


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2830743x65 and 30x22)

In [ ]:
# Define input dimension for AE (L=30)
L = 30 

# Fix incorrect line: Slice to take first 30 columns of Train set
print("Extracting latent features for Train set...")
X_train_latent = autoencoder.extract_features(ae_model, X_train_scaled[:, :L])

# Same for Test 2018 in following cells:
# X_test_latent = autoencoder.extract_features(ae_model, X_test_scaled[:, :L])

Extracting latent features for Train set...


In [ ]:
# Cell 5: Prepare Fusion Data for Train
print("\n--- 4. PREPARE FUSION DATA (TRAIN) ---")
# Get mRMR features
all_feature_names = config.SELECTED_FEATURES
mrmr_indices = [all_feature_names.index(feat) for feat in config.mRMR_FEATURES]

X_train_mrmr = X_train_scaled[:, mrmr_indices]
X_train_fusion = np.hstack([X_train_mrmr, X_train_latent])

print(f"Fusion Train Shape: {X_train_fusion.shape}")

# Delete heavy variables to free RAM for Test set
del X_train, X_train_scaled, df_train
gc.collect()


--- 4. PREPARE FUSION DATA (TRAIN) ---
Fusion Train Shape: (2830743, 25)


3331

In [ ]:
# Cell 6: Train Random Forest on 2017
print("\n--- 5. TRAIN CLASSIFIER (RF) ---")
rf_save_path = exp_paths['models'] / "rf_model_2017.joblib"
rf_model = rf_classifier.train_rf(X_train_fusion, y_train, save_path=rf_save_path)
print("RF Model Trained on 2017.")


--- 5. TRAIN CLASSIFIER (RF) ---
[RandomForest] Training classifier...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100
building tree 4 of 100

building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   51.0s


building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:  3.0min finished


In [ ]:
# Cell 7: Load Data 2018 (TEST)
# ==============================================================================
# MOVE TO TEST PHASE ON 2018
# ==============================================================================
print("\n--- 6. LOADING TEST SET (CSE-CIC-IDS2018) ---")
df_test, y_test = preprocessing.load_single_dataset_year('2018', binary_mode=True)
X_test = df_test.values

print(f"Test Data Shape: {X_test.shape}")


--- 6. LOADING TEST SET (CSE-CIC-IDS2018) ---
🔄 Loading dataset year 2018 (Binary=True)...
✅ Loaded 2018. Shape: (9625148, 65)
Test Data Shape: (9625148, 65)


In [ ]:
# Cell 8: Process Test Data (Use old Scaler & AE)
L = 30  # Ensure L is correct
print("\n--- 7. PROCESSING TEST DATA ---")

# A. Scale (Use 2017 scaler)
print("Scaling using 2017 scaler...")
X_test_scaled = scaler.transform(X_test)

# B. Extract Latent Features (Use 2017 AE)
print("Extracting latent features...")
X_test_latent = autoencoder.extract_features(ae_model, X_test_scaled[:, :L])

# C. Slice mRMR Features
print("Slicing mRMR features...")
X_test_mrmr = X_test_scaled[:, mrmr_indices]

# D. Fusion
print("Fusing...")
X_test_fusion = np.hstack([X_test_mrmr, X_test_latent])
print(f"Fusion Test Shape: {X_test_fusion.shape}")

# Clean RAM
del X_test, X_test_scaled, df_test
gc.collect()


--- 7. PROCESSING TEST DATA ---
Scaling using 2017 scaler...
Extracting latent features...
Slicing mRMR features...
Fusing...
Fusion Test Shape: (9625148, 25)


3019

In [ ]:
# Cell 9: Evaluate Cross-Dataset
print("\n--- 8. FINAL EVALUATION (Train: 2017 -> Test: 2018) ---")
metrics = evaluation.evaluate_model(
    rf_model, 
    X_test_fusion, 
    y_test, 
    save_dir=exp_paths['figures'],
    dataset_name="Cross-Test (Train 17 - Test 18)"
)

print("\n=== RESULTS ===")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"MCC:      {metrics['mcc']:.4f}")


--- 8. FINAL EVALUATION (Train: 2017 -> Test: 2018) ---

📊 Evaluating on Cross-Test (Train 17 - Test 18)...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    6.8s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   16.2s finished


   ✅ Accuracy: 0.7429
   ⭐ MCC:      0.2794
   📝 Report saved to: report_Cross-Test_(Train_17_-_Test_18).txt
   🖼️ Confusion Matrix saved to: cm_Cross-Test_(Train_17_-_Test_18).png

=== KẾT QUẢ ===
Accuracy: 0.7429
MCC:      0.2794


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import gc
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score

# Setup path
sys.path.append(str(Path.cwd().parent))
from src import config, preprocessing, autoencoder, rf_classifier, evaluation, utils

# Create separate folder for Cross-Test 18-17 scenario
exp_paths = utils.setup_experiment_folder()
print(f"Saving results to: {exp_paths['root']}")

print("\n--- 1. LOADING TRAIN SET (CSE-CIC-IDS2018) ---")
# Note: 2018 dataset is very large, ensure machine has sufficient RAM or configure chunking in config
df_train, y_train = preprocessing.load_single_dataset_year('2018', binary_mode=True)

X_train = df_train.values
print(f"Training Data Shape: {X_train.shape}")
print(f"Label Distribution:\n{pd.Series(y_train).value_counts()}")
print("\n--- 2. SCALING ---")
scaler = preprocessing.get_scaler()
# Scaler learns parameters (mean, std) from 2018 set
X_train_scaled = scaler.fit_transform(X_train)
print("Scaler fitted on 2018.")

# Save scaler for later reuse if needed
joblib.dump(scaler, exp_paths['models'] / "scaler_2018.joblib")
print(f"\n--- 3. TRAIN AUTOENCODER (on 2018) ---")

L = 30 # Number of input features (Top-30 mRMR)
N = 5  # Number of latent features (latent space dimension)

# Slice data to take first 30 features for AE
X_train_ae_input = X_train_scaled[:, :L]

ae_model = autoencoder.DeepAutoencoder(
    input_dim=L,
    latent_dim=N,
    hidden_layers=config.AE_HIDDEN_LAYERS # Usually [22, 12]
)

ae_save_path = exp_paths['models'] / "ae_model_2018.pth"
ae_model = autoencoder.train_ae(ae_model, X_train_ae_input, save_path=ae_save_path)

# Extract Latent Features (Z) for training set
print("Extracting latent features for Train set...")
X_train_latent = autoencoder.extract_features(ae_model, X_train_ae_input)
print(f"Latent features extracted: {X_train_latent.shape}")
print("\n--- 4. PREPARE FUSION DATA (TRAIN 2018) ---")

# Get top 20 mRMR features from scaled training set
all_feature_names = config.SELECTED_FEATURES
mrmr_indices = [all_feature_names.index(feat) for feat in config.mRMR_FEATURES]

# IMPORTANT: Use .copy() to ensure data stability after deleting original variable
X_train_mrmr = X_train_scaled[:, mrmr_indices].copy()

# Fusion: [Top-20 mRMR] + [5 Latent Features]
X_train_fusion = np.hstack([X_train_mrmr, X_train_latent])

print(f"Fusion Train Shape: {X_train_fusion.shape}")

# Delete heavy variables to free RAM for 2017 loading
del X_train, X_train_scaled, df_train
gc.collect()
print("\n--- 5. TRAIN CLASSIFIER (RF) ON 2018 ---")
rf_save_path = exp_paths['models'] / "rf_model_2018_fusion.joblib"
rf_model = rf_classifier.train_rf(X_train_fusion, y_train, save_path=rf_save_path)
print("Hybrid RF Model Trained on 2018.")
print("\n--- 6. LOADING TEST SET (CIC-IDS2017) ---")
# Now load 2017 as test set for model trained on 2018
df_test, y_test = preprocessing.load_single_dataset_year('2017', binary_mode=True)
X_test = df_test.values

print(f"Test Data Shape: {X_test.shape}")
print("\n--- 7. PROCESSING TEST DATA (Using 2018 Scaler & AE) ---")

# A. Scale (Use scaler learned from 2018)
X_test_scaled = scaler.transform(X_test)

# B. Extract Latent (Use AE learned from 2018, remember to slice to L=30)
X_test_latent = autoencoder.extract_features(ae_model, X_test_scaled[:, :L])

# C. Slice mRMR Features (Use indices determined earlier)
X_test_mrmr = X_test_scaled[:, mrmr_indices]

# D. Fusion
X_test_fusion = np.hstack([X_test_mrmr, X_test_latent])
print(f"Final Fusion Test Shape: {X_test_fusion.shape}")

# Clean RAM
del X_test, X_test_scaled, df_test
gc.collect()
print("\n--- 8. FINAL EVALUATION (Train: 2018 -> Test: 2017) ---")
metrics = evaluation.evaluate_model(
    rf_model, 
    X_test_fusion, 
    y_test, 
    save_dir=exp_paths['figures'],
    dataset_name="Cross-Test (Train 18 - Test 17)"
)

print("\n=== PERFORMANCE SUMMARY (Proposed Hybrid) ===")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"MCC:      {metrics['mcc']:.4f}")
print(f"AUROC:    {metrics.get('auroc', 'N/A')}")

📂 New Experiment Created: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_083216
📂 Saving results to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_083216

--- 1. LOADING TRAIN SET (CSE-CIC-IDS2018) ---
🔄 Loading dataset year 2018 (Binary=True)...
✅ Loaded 2018. Shape: (9625148, 65)
Training Data Shape: (9625148, 65)
Label Distribution:
Label
0    6876913
1    2748235
Name: count, dtype: int64

--- 2. SCALING ---
✅ Scaler fitted on 2018.

--- 3. TRAIN AUTOENCODER (on 2018) ---
[Autoencoder] Training on cuda...
  Epoch 5/50 - Loss: 0.190192
  Epoch 10/50 - Loss: 0.115097
  Epoch 15/50 - Loss: 0.087841
  Epoch 20/50 - Loss: 0.079465
  Epoch 25/50 - Loss: 0.076470
  Epoch 30/50 - Loss: 0.074279
  Epoch 35/50 - Loss: 0.074590
  Epoch 40/50 - Loss: 0.073321
  Epoch 45/50 - Loss: 0.071115
  Epoch 50/50 - Loss: 0.070748
✅ AE Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_083216\models\ae_model_

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100
building tree 4 of 100

building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  2.8min


building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:  8.9min finished


✅ RF Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_083216\models\rf_model_2018_fusion.joblib
✅ Hybrid RF Model Trained on 2018.

--- 6. LOADING TEST SET (CIC-IDS2017) ---
🔄 Loading dataset year 2017 (Binary=True)...
✅ Loaded 2017. Shape: (2830743, 65)
Test Data Shape: (2830743, 65)

--- 7. PROCESSING TEST DATA (Using 2018 Scaler & AE) ---
Final Fusion Test Shape: (2830743, 25)

--- 8. FINAL EVALUATION (Train: 2018 -> Test: 2017) ---

📊 Evaluating on Cross-Test (Train 18 - Test 17)...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    2.5s finished


   ✅ Accuracy: 0.8296
   ⭐ MCC:      0.3265
   📝 Report saved to: report_Cross-Test_(Train_18_-_Test_17).txt
   🖼️ Confusion Matrix saved to: cm_Cross-Test_(Train_18_-_Test_17).png

=== PERFORMANCE SUMMARY (Proposed Hybrid) ===
Accuracy: 0.8296
MCC:      0.3265
AUROC:    N/A
